In [1]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 75.4 MB/s eta 0:00:00


In [2]:
#Motivated from https://medium.com/@devmallyakarar/quantum-convolutional-neural-networks-for-classification-using-interaction-layers-d94649de42b5
import torch
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import pennylane as qml
from pennylane import numpy as np

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Use native 32x32 CIFAR-10 size
transform = transforms.Compose([
    transforms.ToTensor()
])

# CIFAR-10 Dataset
train_data = datasets.CIFAR10(root='data', train=True, transform=transform, download=True)
test_data = datasets.CIFAR10(root='data', train=False, transform=transform)

train_size = int(0.8 * len(train_data))
val_size = len(train_data) - train_size
train_set, val_set = torch.utils.data.random_split(train_data, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
val_loader = DataLoader(val_set, batch_size=64, shuffle=False)

# Quantum Circuit
n_qubits = 5
n_layers = 3
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

weight_shapes = {"weights": (n_layers, n_qubits)}

# Hybrid Model
class HybridNet(nn.Module):
    def __init__(self):
        super(HybridNet, self).__init__()

        # Feature extractor
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        # Output of 4x4 feature map * 128 channels
        self.fc1 = nn.Linear(128 * 4 * 4, 5)

        # Quantum Layer
        self.qlayer = qml.qnn.TorchLayer(qnode, weight_shapes)

        # Output
        self.fc2 = nn.Linear(5, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))   # 32x16x16
        x = self.pool(F.relu(self.bn2(self.conv2(x))))   # 64x8x8
        x = self.pool(F.relu(self.bn3(self.conv3(x))))   # 128x4x4

        x = x.view(x.size(0), -1)  # Flatten to [batch_size, 128*4*4]
        x = F.relu(self.fc1(x))   # Reduce to 5 features

        x = self.qlayer(x)        # Quantum layer (5 in → 5 out)
        x = self.fc2(x)           # Final layer (5 → 10)
        return x

# Training Setup
model = HybridNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

100%|██████████| 170M/170M [00:03<00:00, 52.0MB/s]


In [4]:
num_epochs=20
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Validation Accuracy: {100 * correct / total:.2f}%")

Epoch 1, Loss: 2.0507
Validation Accuracy: 25.95%
Epoch 2, Loss: 1.7769
Validation Accuracy: 26.77%
Epoch 3, Loss: 1.5319
Validation Accuracy: 42.86%
Epoch 4, Loss: 1.3723
Validation Accuracy: 42.62%
Epoch 5, Loss: 1.2814
Validation Accuracy: 44.84%
Epoch 6, Loss: 1.2041
Validation Accuracy: 49.11%
Epoch 7, Loss: 1.1139
Validation Accuracy: 52.08%
Epoch 8, Loss: 1.0194
Validation Accuracy: 54.24%
Epoch 9, Loss: 0.9365
Validation Accuracy: 51.90%
Epoch 10, Loss: 0.8695
Validation Accuracy: 58.68%
Epoch 11, Loss: 0.8113
Validation Accuracy: 52.89%
Epoch 12, Loss: 0.7590
Validation Accuracy: 59.16%
Epoch 13, Loss: 0.7079
Validation Accuracy: 59.88%
Epoch 14, Loss: 0.6603
Validation Accuracy: 57.87%
Epoch 15, Loss: 0.6331
Validation Accuracy: 55.29%
Epoch 16, Loss: 0.5857
Validation Accuracy: 58.44%
Epoch 17, Loss: 0.5623
Validation Accuracy: 58.81%
Epoch 18, Loss: 0.5371
Validation Accuracy: 57.19%
Epoch 19, Loss: 0.5042
Validation Accuracy: 59.78%
Epoch 20, Loss: 0.4924
Validation Accura